In [1]:
import requests

url = "https://mediadive.dsmz.de/strains"
response = requests.get(url)

if response.status_code == 200:
    print("Successfully fetched the webpage")
else:
    print("Failed to fetch the webpage")

Successfully fetched the webpage


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())  # This will print the HTML structure of the page

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="DSMZ cultivation media database" name="description"/>
  <meta content="Julia Koblitz" name="author"/>
  <!-- Open Graph / Facebook -->
  <meta content="https://mediadive.dsmz.de" property="og:url">
   <meta content="database" property="og:type">
    <meta content="Strains | MediaDive" property="og:title">
     <meta content="Discover standardized cultivation media recipes for more than 40,000 microbial strains." property="og:description">
      <meta content="/img/mediadive_logo.svg" property="og:image"/>
      <!-- Twitter -->
      <meta content="summary_large_image" property="twitter:card"/>
      <meta content="https://mediadive.dsmz.de" property="twitter:url"/>
      <meta content="MediaDive — the cultivation media database" property="twitter:title"/>
      <meta content="Discover standardi